# Twitter did not come back on my Academic research application.
# So I proceeded to work with the Archive analysis from
# https://archive.org/download/archiveteam-twitter-stream-2022-11

In [3]:
# sc master - running locally
sc.master

'local[*]'

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
%pylab inline
import os
import pyspark

# spark is from the previous example.
sc = spark.sparkContext

Populating the interactive namespace from numpy and matplotlib


In [6]:
from dotenv import dotenv_values
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import warnings
import gzip
import json
import os


In [2]:
# Function
def extract_json_from_directory(directory, search_word , col ):
    """Function to unzip and extract all json all files in a given directory"""
    extracted_data = []
    for file_name in os.listdir(directory):
        if file_name.endswith('.json.gz'):
            file_path = os.path.join(directory, file_name)
            with gzip.open(file_path, 'r') as file:
                for line in file:
                    try:
                        json_data = json.loads(line)
                        extracted_data.append(json_data)

                    except json.JSONDecodeError as e:
                        print(f"Error parsing JSON line in {file_name}: {e}")
        df = pd.DataFrame(extracted_data)
        filtered = filter_tweets(df,search_word,col)
    ftdf.append(filtered)
        

    
        #for data in extracted_data:
            #json.dump(data, output)
    return ftdf
directory = "/Volumes/TOSHIBA\ EXT/archive"
search_word = 'trans'
df = extract_json_from_directory(directory,search_word,'text')

# output_file = "/Users/ambrosedesmond/CCT_Projects/Ambrose_MSC_BD_ADA_CA2/Twitter_Archive_Data"

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/TOSHIBA\\ EXT/archive'

In [8]:
df.head()

NameError: name 'df' is not defined

In [5]:
# Function copy to add jsonm search experiment ******************************************works
def extract_json_from_directory(directory, search_words):
    """Function to unzip and extract all json all files in a given directory"""
    extracted_data = []
    my_df = pd.DataFrame()
    for file_name in os.listdir(directory):
        if file_name.endswith('.json.gz'):
            print(f"Extracting file {file_name} ")
            file_path = os.path.join(directory, file_name)
            with gzip.open(file_path, 'r') as file:
                for line in file:
                    try:
                        json_data = json.loads(line)
                        extracted_data.append(json_data)

                    except json.JSONDecodeError as e:
                        print(f"Error parsing JSON line in {file_name}: {e}")
                        
                df = pd.DataFrame(extracted_data)
                filtered_df = df[df["text"].str.contains("gay|trans")]
                # filtered_df = df[df['text'].isin(search_words)]
                
                filtered_df = filtered_df[['created_at','text']]
                my_df = pd.concat([filtered_df,my_df], ignore_index = True)
                # my_df = my_df.append(filtered_df, ignore_index=True)
             
            
            

    return my_df


search_words = ['trans','gay']
directory = "/Volumes/TOSHIBA_EXT/archive"
cbdf = extract_json_from_directory(directory,search_words)
# gtdf = cbdf[['created_at','text']]
gtdf.shape
gtdf.to_csv('Seaerched_tweets_result.csv')

# output_file = "/Users/ambrosedesmond/CCT_Projects/Ambrose_MSC_BD_ADA_CA2/Twitter_Archive_Data"

NameError: name 'gtdf' is not defined

In [45]:
# Set up empty pyspark dataframe to store the imported tweets
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType,TimestampType

spark = SparkSession.builder.getOrCreate() #Start a spark session
# Defining the schema of the pyspark dataframe
schema = StructType([
    StructField("date_extracted", TimestampType(), True),
    StructField("tweet", StringType(), True)
])
stwit_df = spark.createDataFrame([], schema) # make empty dataframe
# show the dataframe is as set out
stwit_df.show()


+--------------+-----+
|date_extracted|tweet|
+--------------+-----+
+--------------+-----+



In [46]:
# Function copy to add gziped and search experiment *************************
# ****** test ************* test *****************
def extract_csv_from_directory(directory, stwit_df):
    """Function to unzip and extract all json all files in a given directory"""

    for file_name in os.listdir(directory):
        if file_name.endswith('.csv.gzip'):
            print(f"Extracting file {file_name} ")
            file_path = os.path.join(directory, file_name)
            with gzip.open(file_path, 'r') as file:
                df = pd.read_csv(file) #read the file as a pandas dataframe
                filter1 = (df['language'] == 'en') & (df['location'] == 'Ukraine') # a filter for english language and location Ukraine
                df = df[filter1] # Selects dataframe based on the filter
                df = df[['extractedts','text']] # outputs only the columns im looking for date and text
                # convert date to datetime object, any error times will be converted to NaT
                df['extractedts'] = pd.to_datetime(df['extractedts'], errors='ignore').dt.date
                df['text'] = df['text'].astype(str) # convert string to string object
                sdf = spark.createDataFrame(df)
                stwit_df = stwit_df.union(sdf)

                        
                #df = pd.DataFrame(extracted_data)
                #filtered_df = df[df["text"].str.contains("gay|trans")]
                # filtered_df = df[df['text'].isin(search_words)]
                
                #filtered_df = filtered_df[['created_at','text']]
                #my_df = pd.concat([filtered_df,my_df], ignore_index = True)
                # my_df = my_df.append(filtered_df, ignore_index=True)
             
            
            

    return stwit_df



directory = "/media/hduser/TOSHIBA_EXT/archive_copy"
cbdf = extract_csv_from_directory(directory,stwit_df)
cbdf.show()


# output_file = "/Users/ambrosedesmond/CCT_Projects/Ambrose_MSC_BD_ADA_CA2/Twitter_Archive_Data"

Extracting file 0401_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0402_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0403_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0404_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0405_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0406_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0407_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0408_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0409_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0410_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0411_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0412_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0413_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0414_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0415_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0416_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 0417_UkraineCombinedTwee

Extracting file UkraineCombinedTweetsDeduped_FEB27.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_FEB28_part1.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_FEB28_part2.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_MAR01.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_MAR02.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_MAR03.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_MAR04.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_MAR05.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_MAR06.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_MAR07.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_MAR08.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_MAR09.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_MAR10.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_MAR11.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_MAR12.csv.gzip 
Extracting file UkraineCombinedTweetsDeduped_MAR13.csv.gzip 
Extracting f

Extracting file 1130_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1201_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1202_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1203_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1204_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1205_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1206_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1207_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1208_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1209_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1210_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1211_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1212_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1213_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1214_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1215_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 1216_UkraineCombinedTwee

Extracting file 20230410_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230411_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230412_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230413_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230414_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230415_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230416_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230417_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230418_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230419_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230420_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230421_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230422_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230423_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230424_UkraineCombinedTweetsDeduped.csv.gzip 
Extracting file 20230425_UkraineCombined

2023-05-23 19:12:32,217 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 2.1 MiB
2023-05-23 19:12:33,257 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 2.1 MiB


+-------------------+--------------------+
|     date_extracted|               tweet|
+-------------------+--------------------+
|2022-04-01 00:00:00|❗️🏫Every school ...|
|2022-04-01 00:00:00|One year ago #Ukr...|
|2022-04-01 00:00:00|JUST IN: #Anonymo...|
|2022-04-01 00:00:00|Chernihiv oblast....|
|2022-04-01 00:00:00|⚡The Ukrainian Ai...|
|2022-04-01 00:00:00|Help to save Ukra...|
|2022-04-01 00:00:00|Help to save Ukra...|
|2022-04-01 00:00:00|✨ Angelina Jolie ...|
|2022-04-01 00:00:00|This is not the t...|
|2022-04-01 00:00:00|#notallrussians??...|
|2022-04-01 00:00:00|⚡The Ukrainian Ai...|
|2022-04-01 00:00:00|🇬🇧🇺🇦How is th...|
|2022-04-01 00:00:00|Grandmother with ...|
|2022-04-01 00:00:00|An activist with ...|
|2022-04-01 00:00:00|⚡The Ukrainian Ai...|
|2022-04-01 00:00:00|The first leader ...|
|2022-04-01 00:00:00|The greatest army...|
|2022-04-01 00:00:00|Ukrainian army so...|
|2022-04-01 00:00:00|1/ Before tomorro...|
|2022-04-01 00:00:00|#Latvia has banne...|
+---------------

In [48]:
print(f'Count of rows:{cbdf.count()}')

2023-05-23 22:08:40,220 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1554.7 KiB


Count of rows:372497


In [49]:
print(f'Count of distinct rows: {cbdf.distinct().count()}')

2023-05-23 22:10:16,722 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 2.0 MiB
2023-05-23 22:11:19,522 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1029.3 KiB


Count of distinct rows: 299821


In [50]:
# There are 72676 duplicate row
cbdf = cbdf.dropDuplicates()

In [51]:
# save the file to Hadoop HDFS
cbdf.write.parquet('hdfs://localhost:9000/user1/Ukraine_twitter.parquet')

2023-05-23 22:34:22,784 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 2.0 MiB
2023-05-23 22:35:22,059 WARN scheduler.DAGScheduler: Broadcasting large task binary with size 1190.3 KiB
2023-05-23 22:35:27,535 WARN hdfs.DataStreamer: Caught exception (42 + 2) / 200]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.hdfs.DataStreamer.closeResponder(DataStreamer.java:986)
	at org.apache.hadoop.hdfs.DataStreamer.endBlock(DataStreamer.java:640)
	at org.apache.hadoop.hdfs.DataStreamer.run(DataStreamer.java:810)


In [7]:
pip install pandas==1.5.3

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 842.5 kB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.0
    Uninstalling pandas-2.0.0:
      Successfully uninstalled pandas-2.0.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
print(pd.__version__)

1.5.3


In [39]:
# file_path = '/home/hduser/Desktop/S2_CA2/0819_UkraineCombinedTweetsDeduped.csv.gzip' 
file_path = '/media/hduser/TOSHIBA_EXT/archive_copy/UkraineCombinedTweetsDeduped_MAR27_to_28.csv.gzip'
with gzip.open(file_path, 'rt') as file: # unzips the file
    df = pd.read_csv(file) #read the file as a pandas dataframe
    filter1 = (df['language'] == 'en') & (df['location'] == 'Ukraine') # a filtersfor english language and location Ukraine
    df = df[filter1] # Selects dataframe based on the filter
    df = df[['extractedts','text']] # outputs only the columns im looking for
    
    
    

df.head()

,extractedts,text
234,2022-03-27 00:13:15.294765,"🇺🇦@ZelenskyyUa : ""I wish I could lend just a f..."
470,2022-03-27 00:13:13.050305,check more #nft\n@opensea\n https://t.co/NHlo1...
569,2022-03-27 00:13:11.064423,@Reuters I would note that it is perfectly acc...
880,2022-03-27 00:12:10.542924,These are not scenes from another fantasy movi...
2341,2022-03-27 00:31:27.710389,Our Anonymous brother @OpsAn0n got suspended ...


In [40]:
print(f'Number of rows with missing values : {df.isnull().any(axis=1).mean()}')

Number of rows with missing values : 0.0


In [41]:
df['extractedts'] = pd.to_datetime(df['extractedts'], errors='raise')

In [35]:
df =df.set_index('extractedts')

In [36]:
df.head()

,text
extractedts,
2022-03-27 00:13:15.294765,"🇺🇦@ZelenskyyUa : ""I wish I could lend just a f..."
2022-03-27 00:13:13.050305,check more #nft\n@opensea\n https://t.co/NHlo1...
2022-03-27 00:13:11.064423,@Reuters I would note that it is perfectly acc...
2022-03-27 00:12:10.542924,These are not scenes from another fantasy movi...
2022-03-27 00:31:27.710389,Our Anonymous brother @OpsAn0n got suspended ...


In [28]:
df.to_csv('failing_date.csv')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3582 entries, 234 to 767845
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   extractedts  3582 non-null   datetime64[ns]
 1   text         3582 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 84.0+ KB


In [12]:
file = '/home/hduser/Desktop/S2_CA2/extracted_data.csv'
df = pd.read_csv(file)
df.head()

,Unnamed: 0,userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetid,...,original_tweet_userid,original_tweet_username,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,is_quote_status,quoted_status_id,quoted_status_userid,quoted_status_username,extractedts
0,0,173212647,JoeMokolobetsi,Yeshua Hamashiach is THE answer | Romans 10:9-...,Afrika Borwa,219,197,4789,2010-07-31 19:09:22.000000,1560416252937617411,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-08-19 08:07:26.836769
1,1,335041409,XclusivasPuebla,Somos el periódico #ExclusivasPuebla| Investi...,"Puebla, México",1419,6402,70267,2011-07-14 02:02:24.000000,1560416256179707904,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-08-19 07:51:50.523048
2,2,1512400441103032323,ShelterAnimalUA,Shelter for abandoned dogs and cats. 1400 dogs...,Ukraine,782,109,1198,2022-04-08 12:02:47.000000,1560416257752666113,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-08-19 05:12:06.194216
3,3,1356632630662430722,DogandCatHelpe1,Shelter for abandoned dogs and cats. 1400 dogs...,Ukraine,5,39,690,2021-02-02 15:57:12.000000,1560416257790382081,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-08-19 11:22:26.824532
4,4,20297125,ElMananaOnline,Las mejores noticias de los dos Laredos y el m...,Nuevo Laredo,2269,17978,56188,2009-02-07 06:32:49.000000,1560416257937051648,...,0,NaN,0,0,NaN,False,0,0,NaN,2022-08-19 11:52:29.448634


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47994 entries, 0 to 47993
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               47994 non-null  int64  
 1   userid                   47994 non-null  int64  
 2   username                 47994 non-null  object 
 3   acctdesc                 43037 non-null  object 
 4   location                 31703 non-null  object 
 5   following                47994 non-null  int64  
 6   followers                47994 non-null  int64  
 7   totaltweets              47994 non-null  int64  
 8   usercreatedts            47994 non-null  object 
 9   tweetid                  47994 non-null  int64  
 10  tweetcreatedts           47994 non-null  object 
 11  retweetcount             47994 non-null  int64  
 12  text                     47994 non-null  object 
 13  hashtags                 47994 non-null  object 
 14  language              

In [25]:
item_count = df['location'].value_counts()
print(item_count)

location
Ukraine                           1233
Україна                            946
Украина                            517
United States                      378
Bay Area, CA                       312
                                  ... 
Novi Sad, Serbia                     1
Atacama, Chile                       1
Over at the Frankenstein Place       1
Birmingham, West Midlands            1
Orange County , CA.                  1
Name: count, Length: 7134, dtype: int64


In [ ]:
# test loading json and searching in one step
import json

def search_json(json_file, search_terms):
    with open(json_file, 'r') as file:
        data = json.load(file)

    results = []
    for item in data:
        text = item.get('text', '')  # Change 'text' to the appropriate key in your JSON structure
        if all(term in text.lower() for term in search_terms):
            results.append(text)

    return results

# Example usage
json_file = 'data.json'  # Replace with your JSON file path
search_terms = ['term1', 'term2', 'term3']  # Replace with your search terms

matching_texts = search_json(json_file, search_terms)
if matching_texts:
    print("Matching texts:")
    for text in matching_texts:
        print(text)
else:
    print("No matching texts found.")
```

Make sure to replace `'text'` with the appropriate key that contains the text string within your JSON structure. Also, update the `json_file` variable with the path to your actual JSON file, and `search_terms` with the set of terms you want to search for.

The script will print all the text strings from the JSON file that contain all the search terms. If no matching texts are found, it will display "No matching texts found."

In [5]:
def load_json_files_and_search(directory):
    combined_results = pd.DataFrame()
    for file_name in os.listdir(directory):
        if file_name.endswith('.json.gz'):
            file_path = os.path.join(directory, file_name)
            df = pd.read_json(file_path)
            search_results = filter_tweets(df,search_word,col)
            combined_results = combined_results.append(search_results, ignore_index=True)
    return combined_results

directory = "/Users/ambrosedesmond/CCT_Projects/Ambrose_MSC_BD_ADA_CA2/Twitter_Archive_Data/20220901"
cbdf = load_json_files_and_search(directory)
cbdf.shape


ValueError: Trailing data

In [10]:
# This function loads the json files one at a time and converts them into dataframe
def convert_json_to_dataframe(json_file):
    """A simple function to convert json file to dataframe"""
    data = []
    with open(json_file, 'r') as file:
        for line in file:
            try:
                json_data = json.loads(line)
                data.append(json_data)
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON line: {e}")

    df = pd.DataFrame(data)
    
    return df
# twit_file = '20220901000000.json'
# Keeping the json files outside my git repository , to reduce git size
#twit_file = "/Users/ambrosedesmond/CCT_Projects/Ambrose_MSC_BD_ADA_CA2/Twitter_Archive_Data/20220901/20220901000000.json"
#df = convert_json_to_dataframe(twit_file)
#df.shape

In [11]:
# path to my zipped tweets
directory = "/Users/ambrosedesmond/CCT_Projects/Ambrose_MSC_BD_ADA_CA2/Twitter_Archive_Data/20220901"
# Call function to extract tweet directory
combined_json = extract_json_from_directory(directory)


TypeError: extract_json_from_directory() missing 2 required positional arguments: 'search_word' and 'col'

In [ ]:
df = convert_json_to_dataframe(combined_json)

In [3]:
def filter_tweets(df, col , search_word):
    filtered_tweets = df[df[col].apply(lambda x: search_word.lower() in x.lower().split())]
    return filtered_tweets


search_word = 'gas'
col = 'text'
fdf = filter_tweets(df, col , search_word)
fdf.shape

NameError: name 'df' is not defined